## Find atomic contacts between antibody and antigen pairs

We want to create a DataFrame `contacts` that contains atomic contacts for all the structures in our cleaned summary file.

The desired result is a DataFrame with columns

- `pdb_id`, e.g. '9ds1'
- `ab_chain`, e.g. 'H' 
- `ab_chaintype`, 'heavy' or 'light'
- `ab_resnum`
- `ab_icode`
- `ab_resname`
- `ab_atom`
- `ag_chain`, e.g. 'G'
- `ag_resnum`
- `ag_icode`
- `ag_resname`, e.g. 'TYR'
- `ag_atom`

We also want to create a DataFrame `residues` that contains occurrence of residues (in order to normalize the contacts).
That DataFrame should contain columns

- `pdb_id`, e.g. '9ds1'
- `ab_chain`, e.g. 'H' 
- `ab_chaintype`, 'heavy' or 'light'
- `ab_resnum`
- `ab_icode`
- `ab_resname`

`residues` contains all residues, while `contacts` only contains residues that have contacts with the antigen chain.

We can accomplish this using our `atomic_contact_points` and `residue_occurrence` functions from notebook 07. Copy this function from the previous notebook into a code cell.

- define empty DataFrame `contacts`
- define empty DataFrame `residues`
- read the cleaned summary file into a pandas DataFrame
- for each row of the summary DataFrame
    - try
        - read the PDB file
        - find atomic contact points for Hchain
        - find atomic contact points for Lchain
        - add columns pbs_id, ab_chain, ab_chaintype ('heavy' and 'light') to the data frames

        - find residues for Hchain
        - find residues for Lchain
        - add columns pbs_id, ab_chain, ab_chaintype ('heavy' and 'light') to the data frames

        - concatenate the  two contact DataFrames to `contacts`
        - concatenate the two residue DataFrames to `residues` 
        
    - except Exception
        - print some error message


Note that we want to put the logic into a try-except block, where we only concatenate to the contacts and residues data frames in the very end. This ensures that we only concatenate to the contacts and residues data frames if everything went fine.

Import required libraries

Define constants

In [ ]:
SUMMARY_FILE = '...'
PDB_DIR = '...'

Create the loop

Save `contacts` and `residues` DataFrames under '../generated/contacts/contacts.tsv' and '../generated/contacts/residues.tsv'.

So in your real repository, this notebook should propbably be somewhere under 'notebooks/contacts'.